# Geneformer Embeddings for Cooper

```
Author: Joshua Pickard
        jpic@umich.edu
Date: September 12, 2024
```

# Set up

In [1]:
import os
import sys
import scanpy as sc
import anndata as ad

import importlib

sys.path.append('../anndata2embedding')
import embed
importlib.reload(embed)
from embed import *

general imports
bioinf imports
scgpt wont work
tGPT will work
geneformer will work
general imports
bioinf imports
scgpt wont work
tGPT will work
geneformer will work


# Load & Chekcout Data

In [2]:
inputdata = [
    '/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/raw.anndata.h5ad',
    '/scratch/indikar_root/indikar1/shared_data/HSC/hsc_landscape.anndata.h5ad'
]

In [3]:
adata1 = sc.read_h5ad(inputdata[0])
adata1.var

,gene_name,Chromosome,Start,End,Strand
gene_id,,,,,
ENSG00000160072,ATAD3B,1,1471764,1497848,+
ENSG00000142611,PRDM16,1,3069167,3438621,+
ENSG00000157933,SKI,1,2228318,2310213,+
ENSG00000142655,PEX14,1,10472287,10630758,+
ENSG00000149527,PLCH2,1,2425979,2505532,+
...,...,...,...,...,...
ENSG00000187191,DAZ3,Y,24763068,24813492,-
ENSG00000188120,DAZ1,Y,23129354,23199010,-
ENSG00000099721,AMELY,Y,6865917,6911752,-


In [6]:
adata2 = sc.read_h5ad(inputdata[1])
adata2.var

,gene_name,ensemble_id,n_counts,n_cells,highly_variable,means,dispersions,dispersions_norm,B_mem,B_naive,...,Mega,Mono,MDSC,Neut,Osteo,Plasma,pDC,Retic,Stromal,cluster_genes
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,A1BG,ENSG00000121410,1653.0,39930,False,0.041392,2.130549,-0.184673,False,False,...,False,False,False,False,False,False,False,False,False,False
A1CF,A1CF,ENSG00000148584,53.0,42149,False,0.001226,0.618202,-0.620396,False,False,...,False,False,False,False,False,False,False,False,False,False
A2M,A2M,ENSG00000175899,471452.0,36875,False,10.314141,20.391302,0.641015,False,False,...,False,False,False,False,False,False,False,False,False,False
A2ML1,A2ML1,ENSG00000166535,6200.0,49012,True,0.392610,8.034155,1.516218,False,False,...,False,False,False,False,False,False,False,False,False,False
A3GALT2,A3GALT2,ENSG00000184389,1324.0,46926,True,0.784540,10.645084,2.268454,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,ZYG11A,ENSG00000203995,129.0,26942,False,0.000869,-0.580755,-0.965828,False,False,...,False,False,False,False,False,False,False,False,False,False
ZYG11B,ZYG11B,ENSG00000162378,28036.0,36992,False,0.225987,0.316922,-0.707198,False,False,...,False,False,False,False,False,False,False,False,False,False
ZYX,ZYX,ENSG00000159840,90723.0,41568,False,0.383848,1.318699,-0.418576,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
print(adata1.shape)
print(adata2.shape)

(10883, 19393)
(54347, 19070)


# Generate embeddings

## sc Fibroblasts

In [13]:
adata1.var['ensemble_id'] = adata1.var.index
adata1.var

,gene_name,Chromosome,Start,End,Strand,ensemble_id
gene_id,,,,,,
ENSG00000160072,ATAD3B,1,1471764,1497848,+,ENSG00000160072
ENSG00000142611,PRDM16,1,3069167,3438621,+,ENSG00000142611
ENSG00000157933,SKI,1,2228318,2310213,+,ENSG00000157933
ENSG00000142655,PEX14,1,10472287,10630758,+,ENSG00000142655
ENSG00000149527,PLCH2,1,2425979,2505532,+,ENSG00000149527
...,...,...,...,...,...,...
ENSG00000187191,DAZ3,Y,24763068,24813492,-,ENSG00000187191
ENSG00000188120,DAZ1,Y,23129354,23199010,-,ENSG00000188120
ENSG00000099721,AMELY,Y,6865917,6911752,-,ENSG00000099721


In [14]:
import numpy as np
adata1.obs['n_counts_UMIs'] = np.sum(adata1.X, axis=1)
adata1.obs

,n_genes,n_counts_UMIs
cell_id,,
AAACCAAAGGGTAGCA,16001,16001
AAACCAAAGTAAGGGT,7864,7864
AAACCATTCAGGTAGG,15610,15610
AAACCATTCCAGCCCT,10505,10505
AAACCATTCGTGACCG,8775,8775
...,...,...
TGTGGTTGTAGCCCTA,10412,10412
TGTGGTTGTTGAAGGC,11811,11811
TGTGTTAGTTGGTAAG,6249,6249


In [15]:
output_path = '/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/'
output_file = 'geneformer_no_perturbed_jpic_9_12_2024.h5ad'
# Embed the AnnData object using Geneformer
embedAdGF = embed(adata1, 'geneformer',
                  output_directory=output_path,
                  output_file=output_file,
                  genenameloc='ensemble_id',
                  n_counts_column='n_counts_UMIs'
                 )



output_directory='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/'
l212: output_path='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/'
l213: type(output_path)=<class 'str'>
l214: (output_path==None)=False
Tokenizing the dataset


/home/jpic/.local/lib/python3.11/site-packages/anndata/_core/views.py:144: RuntimeWarning: invalid value encountered in divide
  results = super().__array_ufunc__(


Map (num_proc=16):   0%|          | 0/10883 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10883 [00:00<?, ? examples/s]

/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


/home/jpic/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/1089 [00:00<?, ?it/s]

In [16]:
embedAdGF.obs = adata1.obs

In [18]:
embedAdGF.obs

,n_genes,n_counts_UMIs
cell_id,,
AAACCAAAGGGTAGCA,16001,16001
AAACCAAAGTAAGGGT,7864,7864
AAACCATTCAGGTAGG,15610,15610
AAACCATTCCAGCCCT,10505,10505
AAACCATTCGTGACCG,8775,8775
...,...,...
TGTGGTTGTAGCCCTA,10412,10412
TGTGGTTGTTGAAGGC,11811,11811
TGTGTTAGTTGGTAAG,6249,6249


In [21]:
embedAdGF.write_h5ad(os.path.join(output_path, output_file))

## HSC landscape embeddings

In [22]:
adata2.var

,gene_name,ensemble_id,n_counts,n_cells,highly_variable,means,dispersions,dispersions_norm,B_mem,B_naive,...,Mega,Mono,MDSC,Neut,Osteo,Plasma,pDC,Retic,Stromal,cluster_genes
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,A1BG,ENSG00000121410,1653.0,39930,False,0.041392,2.130549,-0.184673,False,False,...,False,False,False,False,False,False,False,False,False,False
A1CF,A1CF,ENSG00000148584,53.0,42149,False,0.001226,0.618202,-0.620396,False,False,...,False,False,False,False,False,False,False,False,False,False
A2M,A2M,ENSG00000175899,471452.0,36875,False,10.314141,20.391302,0.641015,False,False,...,False,False,False,False,False,False,False,False,False,False
A2ML1,A2ML1,ENSG00000166535,6200.0,49012,True,0.392610,8.034155,1.516218,False,False,...,False,False,False,False,False,False,False,False,False,False
A3GALT2,A3GALT2,ENSG00000184389,1324.0,46926,True,0.784540,10.645084,2.268454,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,ZYG11A,ENSG00000203995,129.0,26942,False,0.000869,-0.580755,-0.965828,False,False,...,False,False,False,False,False,False,False,False,False,False
ZYG11B,ZYG11B,ENSG00000162378,28036.0,36992,False,0.225987,0.316922,-0.707198,False,False,...,False,False,False,False,False,False,False,False,False,False
ZYX,ZYX,ENSG00000159840,90723.0,41568,False,0.383848,1.318699,-0.418576,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
adata2.obs

,n_counts,cell_type,dataset,organ_tissue,n_genes,new_cluster,cell_label,cluster_str,UMAP 1,UMAP 2
AAACCCAAGGTTACCT_iHSC,6558.0,iHSC,iHSC,nan,12480,1,Reprogrammed Cells,D2,8.989130,4.937538
AAACCCAAGTTGAAGT_iHSC,5488.0,iHSC,iHSC,nan,12413,1,Reprogrammed Cells,D2,7.586400,4.889987
AAACCCAAGTTGTCGT_iHSC,4330.0,iHSC,iHSC,nan,11906,10,Reprogrammed Cells,D11,8.027761,3.311177
AAACCCACAGAAGCGT_iHSC,3442.0,iHSC,iHSC,nan,11861,10,Reprogrammed Cells,D11,8.584245,2.674468
AAACCCACAGGAGGTT_iHSC,14427.0,iHSC,iHSC,nan,13436,1,Reprogrammed Cells,D2,9.509144,4.923519
...,...,...,...,...,...,...,...,...,...,...
TTTGACTTCGCAATCG-2_old1_BMMC_HSPC,3616.0,MPP,old1_BMMC_HSPC,nan,13075,8,Multipotent Progenitor,D9,5.472989,7.853297
TTTGACTTCGTTAAGC-2_old1_BMMC_HSPC,1262.0,MPP,old1_BMMC_HSPC,nan,12312,2,Multipotent Progenitor,D3,4.184635,4.982112
TTTGAGTCAAGGAATC-2_old1_BMMC_HSPC,4720.0,MPP,old1_BMMC_HSPC,nan,13563,2,Multipotent Progenitor,D3,4.448498,7.588126
TTTGGTGCAGAGAGCC-2_old1_BMMC_HSPC,5012.0,MPP,old1_BMMC_HSPC,nan,13628,2,Multipotent Progenitor,D3,4.706429,7.555507


In [24]:
output_path = '/scratch/indikar_root/indikar1/shared_data/HSC/'
output_file = 'geneformer_no_perturbed_jpic_9_12_2024.h5ad'
# Embed the AnnData object using Geneformer
embedAdGF = embed(adata2, 'geneformer',
                  output_directory=output_path,
                  output_file=output_file,
                  genenameloc='ensemble_id',
                  n_counts_column='n_counts'
                 )

output_directory='/scratch/indikar_root/indikar1/shared_data/HSC/'
l212: output_path='/scratch/indikar_root/indikar1/shared_data/HSC/'
l213: type(output_path)=<class 'str'>
l214: (output_path==None)=False
Tokenizing the dataset


Map (num_proc=16):   0%|          | 0/54347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/54347 [00:00<?, ? examples/s]

/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


  0%|          | 0/5435 [00:00<?, ?it/s]

In [26]:
embedAdGF.obs = adata2.obs
embedAdGF.obs

,n_counts,cell_type,dataset,organ_tissue,n_genes,new_cluster,cell_label,cluster_str,UMAP 1,UMAP 2
AAACCCAAGGTTACCT_iHSC,6558.0,iHSC,iHSC,nan,12480,1,Reprogrammed Cells,D2,8.989130,4.937538
AAACCCAAGTTGAAGT_iHSC,5488.0,iHSC,iHSC,nan,12413,1,Reprogrammed Cells,D2,7.586400,4.889987
AAACCCAAGTTGTCGT_iHSC,4330.0,iHSC,iHSC,nan,11906,10,Reprogrammed Cells,D11,8.027761,3.311177
AAACCCACAGAAGCGT_iHSC,3442.0,iHSC,iHSC,nan,11861,10,Reprogrammed Cells,D11,8.584245,2.674468
AAACCCACAGGAGGTT_iHSC,14427.0,iHSC,iHSC,nan,13436,1,Reprogrammed Cells,D2,9.509144,4.923519
...,...,...,...,...,...,...,...,...,...,...
TTTGACTTCGCAATCG-2_old1_BMMC_HSPC,3616.0,MPP,old1_BMMC_HSPC,nan,13075,8,Multipotent Progenitor,D9,5.472989,7.853297
TTTGACTTCGTTAAGC-2_old1_BMMC_HSPC,1262.0,MPP,old1_BMMC_HSPC,nan,12312,2,Multipotent Progenitor,D3,4.184635,4.982112
TTTGAGTCAAGGAATC-2_old1_BMMC_HSPC,4720.0,MPP,old1_BMMC_HSPC,nan,13563,2,Multipotent Progenitor,D3,4.448498,7.588126
TTTGGTGCAGAGAGCC-2_old1_BMMC_HSPC,5012.0,MPP,old1_BMMC_HSPC,nan,13628,2,Multipotent Progenitor,D3,4.706429,7.555507


In [27]:
embedAdGF.write_h5ad(os.path.join(output_path, output_file))

## Perturbe and Embed

In [4]:
adata1.var['ensemble_id'] = adata1.var.index
adata1.var

,gene_name,Chromosome,Start,End,Strand,ensemble_id
gene_id,,,,,,
ENSG00000160072,ATAD3B,1,1471764,1497848,+,ENSG00000160072
ENSG00000142611,PRDM16,1,3069167,3438621,+,ENSG00000142611
ENSG00000157933,SKI,1,2228318,2310213,+,ENSG00000157933
ENSG00000142655,PEX14,1,10472287,10630758,+,ENSG00000142655
ENSG00000149527,PLCH2,1,2425979,2505532,+,ENSG00000149527
...,...,...,...,...,...,...
ENSG00000187191,DAZ3,Y,24763068,24813492,-,ENSG00000187191
ENSG00000188120,DAZ1,Y,23129354,23199010,-,ENSG00000188120
ENSG00000099721,AMELY,Y,6865917,6911752,-,ENSG00000099721


In [5]:
def iterate_perturb_id_counts(adata, tf_list, scalar_list):
    """
    Applies perturbations to the expression data of specified transcription factors across multiple scalars 
    and stores the resulting AnnData objects in a dictionary.

    This function performs the following steps:
    1. Iterates over a list of scalar values.
    2. For each scalar, creates a copy of the AnnData object to preserve the original data.
    3. Applies the `perturb_counts` function to scale the expression data of genes listed in `tf_list` by
       the maximum gene expression of each cell and the current scalar.
    4. Stores the perturbed AnnData object in a dictionary with the scalar as the key.

    Parameters:
    tf_list (list): A list of gene symbols (transcription factors) to be perturbed.
    scalar_list (list): A list of scalar values for scaling the gene expression.
    adata (AnnData): The AnnData object containing gene expression data (cells x genes).

    Returns:
    dict: A dictionary where keys are scalar values and values are the corresponding perturbed AnnData objects.
    """
    
    adata_dict = {}
    
    for scalar in scalar_list:
        # Create a copy of the AnnData object for each scalar value
        adata_temp = adata.copy()
        
        # Apply perturb_counts to the copied AnnData object
        perturbed_adata = perturb_id_counts(adata_temp, tf_list, scalar) # JP change this line
        
        # Store the perturbed AnnData object in the dictionary with scalar as the key
        adata_dict[scalar] = perturbed_adata
    
    return adata_dict

def perturb_id_counts(adata, tf_list, scalar): 
    """
    Applies a perturbation to the expression data of specific genes in an AnnData object.

    This function performs the following steps:
    1. Computes the maximum gene expression level for each cell.
    2. Applies a scaling operation to the expression levels of genes listed in `tf_list`.
       - Each entry of these genes in the matrix is multiplied by the maximum expression level 
         of its respective cell and a specified scalar value.
    3. Updates the AnnData object with new columns:
       - 'scaled': A boolean column indicating whether each gene is in the `tf_list`.
       - 'scaled_by': Contains the scaling factor used for each gene (the product of the maximum 
         expression level of each cell and the scalar), or `1` if the gene was not in `tf_list`.
    
    Parameters:
    tf_list (list): A list of gene symbols to be perturbed.
    scalar (float): The scalar value used to scale the expression levels.
    adata (AnnData): The AnnData object containing gene expression data.

    Returns:
    AnnData: The updated AnnData object with applied perturbations and new columns.
    """

    # Create a boolean mask for genes in tf_list
    gene_mask = adata.var['gene_name'].isin(tf_list)
    
    # Save the original state of the parameter objects, in case some tfs do not translate (failsafe)
    original_X = adata.X.copy()
    original_gene_mask = gene_mask.copy()
    
    # Compute maximum expression level of each cell
    max_exp = np.max(adata.X, axis=1)

    """This is new today. v """
    # Raise an error if any of the gene names in tf_list do not match column names (we will manually update these in adata):
    missing_genes = [gene for gene in tf_list if gene not in adata.var['gene_name'].values]
    
    if missing_genes:
        # Restore original parameter objects
        adata.X = original_X
        gene_mask = original_gene_mask
        raise ValueError(f"Genes {missing_genes} not found in anndata object")

    else:    
        
        # Apply the scaling operation to the specified genes
        adata.X[:, gene_mask] = (max_exp * scalar).reshape(-1, 1)
        
        # Add/Update 'scaled' column in var
        adata.var['scaled'] = gene_mask
        
        # Add/Update 'scaled_by' column in var
        adata.var['scaled_by'] = scalar  # Default value for genes not in tf_list

        # Add/Update 'scaled_by' column in var
        adata.obs['U'] = scalar  # Default value for genes not in tf_list
    # adata.var = 
    return adata

In [6]:
output_directory = "/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/"
TFlist = ['GATA2', 'GFI1B', 'STAT5A', 'REL', 'FOS']
scalars = [0.25, 0.5, 0.75, 1.001]

TFcombos = [[tf] for tf in TFlist]
TFcombos.append(TFlist)
TFcombos

[['GATA2'],
 ['GFI1B'],
 ['STAT5A'],
 ['REL'],
 ['FOS'],
 ['GATA2', 'GFI1B', 'STAT5A', 'REL', 'FOS']]

In [35]:
TFs = ['GATA2']

In [36]:
"_".join(TFs)

'GATA2'

In [7]:
for TFs in TFcombos:
    TFs_str = str(TFs)
    if isinstance(TFs, list):
        TFs_str = "_".join(TFs)
    output_path = os.path.join(output_directory, f"{TFs_str}.h5ad")
    
    # Check if the file already exists
    if os.path.exists(output_path):
        print(f"{output_path} already exists: continuing to next TF!")
        continue

    print(f"{output_path=}")

    print(f"Perturbing TFs: {TFs}")
    adataDict = iterate_perturb_id_counts(adata1.copy(), TFs, scalars)

    # Concatenate all AnnData objects along the observations axis
    concatenated_adata = ad.concat(list(adataDict.values()), axis=0)

    # Copy var information from the original AnnData
    concatenated_adata.var = adata1.var.copy()

    # Print before embeddings
    print('Beginning geneformer embeddings')
    
    embedAdGF = embed(concatenated_adata, 'geneformer',
                      output_directory=output_path,
                      output_file=output_path,
                      genenameloc='ensemble_id',
                      n_counts_column='n_counts'
                     )
    
    # Save the concatenated AnnData object to the file
    embedAdGF.write_h5ad(output_path)

/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GATA2.h5ad already exists: continuing to next TF!
/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GFI1B.h5ad already exists: continuing to next TF!
/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/STAT5A.h5ad already exists: continuing to next TF!
/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/REL.h5ad already exists: continuing to next TF!
output_path='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/FOS.h5ad'
Perturbing TFs: ['FOS']


/home/jpic/.local/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Beginning geneformer embeddings
output_directory='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/FOS.h5ad'
l212: output_path='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/FOS.h5ad'
l213: type(output_path)=<class 'str'>
l214: (output_path==None)=False
Dataset is pretokenized
/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


/home/jpic/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/4354 [00:00<?, ?it/s]

...

/home/jpic/.local/lib/python3.11/site-packages/anndata/_core/views.py:144: RuntimeWarning: invalid value encountered in divide
  results = super().__array_ufunc__(


Map (num_proc=16):   0%|          | 0/43532 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/43532 [00:00<?, ? examples/s]

/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


  0%|          | 0/4354 [00:00<?, ?it/s]

In [8]:
adata = sc.read_h5ad('/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GATA2_GFI1B_STAT5A_REL_FOS.h5ad')

/home/jpic/.local/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [56]:
embedAdGF = embed(concatenated_adata, 'geneformer',
                  output_directory=output_path,
                  output_file=output_path,
                  genenameloc='ensemble_id',
                  n_counts_column='n_counts'
                 )

output_directory='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GATA2.h5ad'
l212: output_path='/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GATA2.h5ad'
l213: type(output_path)=<class 'str'>
l214: (output_path==None)=False
Tokenizing the dataset


/home/jpic/.local/lib/python3.11/site-packages/anndata/_core/views.py:144: RuntimeWarning: invalid value encountered in divide
  results = super().__array_ufunc__(


Map (num_proc=16):   0%|          | 0/43532 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/43532 [00:00<?, ? examples/s]

/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/


  0%|          | 0/4354 [00:00<?, ?it/s]

In [58]:
# Save the concatenated AnnData object to the file
embedAdGF.write_h5ad(output_path)

In [57]:
output_path

'/scratch/indikar_root/indikar1/shared_data/single_cell_fibroblast/scanpy/GATA2.h5ad'

In [48]:
concatenated_adata.var = adata1.var.copy()

In [54]:
concatenated_adata.var

,gene_name,Chromosome,Start,End,Strand,ensemble_id
gene_id,,,,,,
ENSG00000160072,ATAD3B,1,1471764,1497848,+,ENSG00000160072
ENSG00000142611,PRDM16,1,3069167,3438621,+,ENSG00000142611
ENSG00000157933,SKI,1,2228318,2310213,+,ENSG00000157933
ENSG00000142655,PEX14,1,10472287,10630758,+,ENSG00000142655
ENSG00000149527,PLCH2,1,2425979,2505532,+,ENSG00000149527
...,...,...,...,...,...,...
ENSG00000187191,DAZ3,Y,24763068,24813492,-,ENSG00000187191
ENSG00000188120,DAZ1,Y,23129354,23199010,-,ENSG00000188120
ENSG00000099721,AMELY,Y,6865917,6911752,-,ENSG00000099721


In [41]:
# adata1.var['ensemble_id'] = adata1.var.index
adata1.var

,gene_name,Chromosome,Start,End,Strand,ensemble_id
gene_id,,,,,,
ENSG00000160072,ATAD3B,1,1471764,1497848,+,ENSG00000160072
ENSG00000142611,PRDM16,1,3069167,3438621,+,ENSG00000142611
ENSG00000157933,SKI,1,2228318,2310213,+,ENSG00000157933
ENSG00000142655,PEX14,1,10472287,10630758,+,ENSG00000142655
ENSG00000149527,PLCH2,1,2425979,2505532,+,ENSG00000149527
...,...,...,...,...,...,...
ENSG00000187191,DAZ3,Y,24763068,24813492,-,ENSG00000187191
ENSG00000188120,DAZ1,Y,23129354,23199010,-,ENSG00000188120
ENSG00000099721,AMELY,Y,6865917,6911752,-,ENSG00000099721


In [51]:
concatenated_adata.obs

,n_genes,U
cell_id,,
AAACCAAAGGGTAGCA,16001,0.250
AAACCAAAGTAAGGGT,7864,0.250
AAACCATTCAGGTAGG,15610,0.250
AAACCATTCCAGCCCT,10505,0.250
AAACCATTCGTGACCG,8775,0.250
...,...,...
TGTGGTTGTAGCCCTA,10412,1.001
TGTGGTTGTTGAAGGC,11811,1.001
TGTGTTAGTTGGTAAG,6249,1.001
